In [21]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.compat.v1.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data', one_hot=True)

n_noise =100
n_h1=150
n_h2=300
batch_size=256
n_epoch=50000

def generator(z_noise):
    w1 = tf.Variable(tf.truncated_normal([n_noise, n_h1], stddev=0.1), name="w1_g", dtype=tf.float32)
    b1=tf.Variable(tf.zeros([n_h1]),name="b1_g",dtype=tf.float32)
    h1=tf.nn.ReLU(tf.matmul(z_noise, w1) +b1)
    w2 = tf.Variable(tf.truncated_normal([n_h1,n_h2], stddev=0.1), name = "w2_g", dtype=tf.float32)
    b2=tf.Variable(tf.zeros([n_h2]),name="b2_g",dtype=tf.float32)
    h2=tf.nn.ReLU(tf.matmul(h1, w2) +b2)
    w3 = tf.Variable(tf.truncated_normal([n_h2,28*28], stddev=0.1), name = "w3_g", dtype=tf.float32)
    b3=tf.Variable(tf.zeros([28*28]),name="b3_g",dtype=tf.float32)
    h3=tf.matmul(h2, w3) +b3
    out_gen=tf.nn.tanh(h3)
    weight_g = [w1,b1,w2,b2,w3,b3]
    return out_gen, weight_g

def discriminator(x,out_gen,keep_prob):
    x_all = tf.concat([x,out_gen], 0)
    w1=tf.Variable(tf.truncated_normal([28*28,n_h2], stddev=0.1), name="w1_d", dtype=tf.float32)
    b1=tf.Variable(tf.zeros([n_h2]),name="b1_d",dtype=tf.float32)
    h1=tf.nn.dropout(tf.nn.ReLU(tf.matmul(x_all, w1) +b1),keep_prob)
    w2 = tf.Variable(tf.truncated_normal([n_h2,n_h1], stddev=0.1), name = "w2_d", dtype=tf.float32)
    b2=tf.Variable(tf.zeros([n_h1]),name="b2_d",dtype=tf.float32)
    h2=tf.nn.dropout(tf.nn.ReLU(tf.matmul(h1, w2) +b2), keep_prob)
    w3 = tf.Variable(tf.truncated_normal([n_h1,1], stddev=0.1), name = "w3_d", dtype=tf.float32)
    b3=tf.Variable(tf.zeros([1]),name="d_b3",dtype=tf.float32)
    h3=tf.matmul(h2, w3) +b3
    y_data = tf.nn.sigmoid(tf.slice(h3,[0,0],[batch_size, -1], name=None))
    y_fake = tf.nn.sigmoid(tf.slice(h3,[batch_size, 0],[-1,-1], name=None))
    weight_d =[w1,b1,w2,b2,w3,b3]
    return y_data, y_fake, weight_d

x= tf.placeholder(tf.float32, [batch_size, 28*28], name="x_data")
z_noise = tf.placeholder(tf.float32, [batch_size, n_noise], name="z_prior")

keep_prob= tf.placeholder(tf.float32, name="keep_prob")

out_gen, weight_g = generator(z_noise)
y_data, y_fake, weight_d = discriminator(x, out_gen, keep_prob)

disc_loss = -(tf.log(y_data) + tf.log(1-y_fake))
gen_loss = -tf.log(y_fake)
optimizer = tf.train.AdamOptimizer(0.0001)
d_trainer = optimizer.minimize(disc_loss, var_list = weight_d)
g_trainer = optimizer.minimize(gen_loss,var_list=weight_g)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
z_sample = np.random.uniform(-1,1,size=(batch_size, n_noise)).astype(np.float32)

for epoch in range(n_epoch):
    batch_x, _= mnist.train.next_batch(batch_size)
    x_value = 2*batch_x.astype(np.float32) -1
    z_value = np.random.uniform(-1,1,size=(batch_size, n_noise)).astype(np.float32)
    sess.run(d_trainer, feed_dict={x:x_value, z_noise:z_value,keep_prob:0.7})
    sess.run(g_trainer, feed_dict={x:x_value, z_noise:z_value,keep_prob:0.7})
    [c1,c2] = sess.run([disc_loss, gen_loss], feed_dict = {x:x_value, z_noise:z_value, keep_prob:0.7})
    print("on epoch{}".format(epoch))
    
out_img= sess.run(out_gen, feed_dict={z_noise:z_sample})
imgs=0.5*(out_img +1)

for k in range(25):
    plt.subplot(5,5,k+1)
    image = np.reshape(imgs[k],(28,28))
    plt.imshow(image, cmap='gray')

ModuleNotFoundError: No module named 'tensorflow.compat.v1.examples'